## 200. Number of Islands
- Description:
  <blockquote>
    Given an m x n 2D binary grid grid which represents a map of '1's (land) and '0's (water), return the number of islands.

    An island is surrounded by water and is formed by connecting adjacent lands horizontally or vertically. You may assume all four edges of the grid are all surrounded by water.

    

    Example 1:

    Input: grid = [
      ["1","1","1","1","0"],
      ["1","1","0","1","0"],
      ["1","1","0","0","0"],
      ["0","0","0","0","0"]
    ]
    Output: 1

    Example 2:

    Input: grid = [
      ["1","1","0","0","0"],
      ["1","1","0","0","0"],
      ["0","0","1","0","0"],
      ["0","0","0","1","1"]
    ]
    Output: 3

    

    Constraints:

        m == grid.length
        n == grid[i].length
        1 <= m, n <= 300
        grid[i][j] is '0' or '1'.
  </blockquote>

- URL: [Problem_URL](https://leetcode.com/problems/number-of-islands/description/)

- Topics: Graph, DFS, BFS, Union Find

- Difficulty: Medium

- Resources: example_resource_URL

### Solution 1, Recursive DFS, marking visited cells by changing them to '#'

R - rows, C- cols

- Time Complexity: O(R * C)
- Space Complexity: O(R * C)
  -  in case that the grid map is filled with lands where DFS goes by R×C deep.

In [ ]:
from typing import List


class Solution:
    def numIslands(self, grid: List[List[str]]) -> int:
        result = 0
        row_len = len(grid)
        col_len = len(grid[0])
        DIRECTIONS = ((-1, 0), (1, 0), (0, -1), (0, 1))


        def valid_cell(rw, cl):
            return 0 <= rw < row_len and 0 <= cl < col_len

        def dfs(rw, cl):
            # If row and/or col values are outside of grid bounds return
            if not valid_cell(rw, cl):
                return
            
            # Check if cell is water or is already visited AKA anything but land
            if grid[rw][cl] != "1":
                return

            # Mark cell as visited by setting the cells value to '#'
            grid[rw][cl] = "#"

            # Visit adjacent cells in valid directions
            for dr, dc in DIRECTIONS:
                nr = rw+dr
                nc = cl+dc

                dfs(nr, nc)
            
        
        for row in range(row_len):
            for col in range(col_len):
                # If cell contains land, visit neighboring cells in dfs manner 
                # to find connected land cells that make up a single island
                if grid[row][col] == "1":
                    dfs(row, col)
                    result += 1
        
        return result

### Solution 1.1, Recursive DFS using Visited Set to mark visited (row, col) tuples
Solution description
- Time Complexity: O(N)
- Space Complexity: O(N)

In [ ]:
class Solution:
    def numIslands(self, grid: List[List[str]]) -> int:
        if not grid:
            return 0
        
        DIRECTIONS = ((-1, 0), (1, 0), (0, -1), (0, 1))
        island_count = 0
        visited = set()

        def dfs(rw, cl):
            # Check row and col are within grid bounds
            if not (0 <= rw < len(grid)) or not (0 <= cl < len(grid[0])):
                return

            # Check if cell contains land
            if grid[rw][cl] != "1":
                return
            
            # Check if cells has been visited
            if (rw, cl) in visited:
                return
            
            # Mark cell as visited
            visited.add((rw, cl))

            # Visit adjacent cells vertically and horizontally
            for dr, dc in DIRECTIONS:
                nr = rw+dr
                nc = cl+dc

                dfs(nr, nc)

        for rw in range(len(grid)):
            for cl in range(len(grid[0])):
                # If cell contains land, visit neighboring cells (horizontally or vertically) in dfs manner
                # to find connected land cells that make up a single island
                if grid[rw][cl] == "1" and (rw, cl) not in visited:
                    dfs(rw, cl)
                    island_count += 1
        
        return island_count

### Solution 2, Most Optimum Space Complexity, BFS by marking visited cells by changing them to '#'
R - rows, C- cols

- Time Complexity: O(R * C)
- Space Complexity: O(min(R,C))
  - Because in worst case where the grid is filled with lands, the size of queue can grow up to min(M,N)
  - Maximum siblings in queue will be min(M, N)
  - [visual explanation](https://imgur.com/gallery/bfs-2d-grid-of-m-x-n-M58OKvB)

In [ ]:
from collections import deque


class Solution:
    def numIslands(self, grid: List[List[str]]) -> int:
            if not grid:
                return 0
            
            DIRECTIONS = ((-1, 0), (1, 0), (0, -1), (0, 1))
            
            def valid_cell(rw, cl):
                return 0 <= rw < len(grid) and 0 <= cl < len(grid[0])

            def bfs(rw, cl):
                queue = deque()
                queue.append((rw, cl))

                # Mark cell as visited by setting the cells value to '#'
                grid[rw][cl] = '#'

                while queue:
                    rw, cl = queue.popleft()

                    for dr, dc in DIRECTIONS:
                        nr = rw+dr
                        nc = cl+dc
                        
                        # If row and col are within grid bounds and current cell is a land cell, Visit adjacent cells vertically and horizontally
                        if valid_cell(nr, nc) and grid[nr][nc] == '1':
                            # In BFS, if you only mark a cell as visited after you pop it from the queue, you run the risk of adding the same cell to the queue multiple times.
                            grid[nr][nc] = '#'
                            queue.append((nr, nc))


            
            island_count = 0
            for rw in range(len(grid)):
                for cl in range(len(grid[0])):
                    # If cell contains land, visit neighboring cells (horizontally or vertically) in bfs manner
                    # to find connected land cells that make up a single island
                    if grid[rw][cl] == "1":
                        bfs(rw, cl)
                        island_count += 1
            
            return island_count

### Solution 3, Union Find AKA Disjoint Set Union (DSU) Approach

#### When to use this approach:
Best for Dynamic Grids (where land is added over time)

Imagine the grid starts as all water and land is added one cell at a time. With BFS, you’d have to restart the whole search every time land is added. With Union-Find, you just perform a single union() operation with the new cell’s neighbors. It's much more efficient for real-time updates!

---

Only Two Directions: Notice we only check Right and Down. When you are iterating through every cell, checking "forward" is sufficient because the "backward" connections (Left and Up) were already handled when we were at those previous cells.

The Initialization: The uf.count starts at the total number of land cells. Every time a union actually merges two separate islands into one, we decrement that count.

Path Compression: In the find method, the line self.parent[i] = self.find(self.parent[i]) is the "secret sauce." It flattens the tree structure every time it's called, making future lookups nearly instantaneous.

### uf.union(r * cols + c, nr * cols + nc):
Since the Union-Find parent array is a simple list like [0, 1, 2, 3, ...], it doesn't understand coordinates like (row,col). We have to give every cell a unique ID from 0 to (rows×cols)−1.
To turn a 2D coordinate into a 1D index, we use:
Index=(current_row×total_columns)+current_column

Imagine a 3×4 grid (3 rows, 4 columns):

|  | Col 0 | Col 1 | Col 2 | Col 3 |
| --- | --- | --- | --- | --- |
| **Row 0** | (0,0) | (0,1) | (0,2) | (0,3) |
| **Row 1** | (1,0) | (1,1) | (1,2) | (1,3) |
| **Row 2** | (2,0) | (2,1) | (2,2) | (2,3) |

Export to Sheets

If we want the 1D index for the cell at **Row 1, Col 2**:

1.  We have already completed **1 full row** (Row 0).
    
2.  Each row has **4 columns**.
    
3.  So, 1 row×4 columns\=4 cells passed.
    
4.  Now, add the current column index (2).
    
5.  **Result:** 4+2\=6.
    

If you count the cells starting from the top left (0, 1, 2, 3, 4, 5...), the cell at (1, 2) is indeed the **6th** index!

🔄 Can you go backwards?

Yes! If you have the 1D index and want the 2D coordinates back, you use the modulo and division operators:
row=index//cols
col=index%cols

### Why do this 3D to 2D mapping
Python handles 1D lists and 2D lists (lists of lists) very differently in memory:

- 1D List: A single contiguous block of memory. Accessing parent[i] is a single, lightning-fast pointer jump.

- 2D List: A "list of pointers to other lists." To get to parent[r][c], Python has to:

  1. Find the outer list.

  2. Follow a pointer to the r-th inner list.

  3. Follow a pointer to the c-th element.

This "double lookup" adds overhead. In a problem like Number of Islands, where you might call find thousands of times, those extra nanoseconds add up.

Also a 2D array for parent and rank would make the code more verbose and complex


When you use Path Compression (the trick where you point nodes directly to the root during a find), the "Rank" is no longer the literal height of the tree—it becomes an upper bound on the height. This is why it's called "Rank" instead of "Height."

Imagine two trees, Tree A and Tree B, both with a height (rank) of 2.

    Tree A root: RA​

    Tree B root: RB​

    The Union: We decide to make RA​ the parent of RB​.

    The Result: RB​ is now one level below RA​.

Since RB​ was already the top of a tree with height 2, and it is now sitting underneath RA​, the total path from RA​ down t

R - rows, C- cols  
The actual time complexity is O(M×N×α(M×N)), where α is the Inverse Ackermann function—which is so slow-growing it's effectively constant.  
- Time Complexity: O(R * C)
  - Union operation takes essentially constant time when UnionFind is implemented with both path compression and union by rank.
- Space Complexity: O(R * C)
  - as required by UnionFind data structure

In [ ]:
# Gemini Union Find sol, # Union by Rank approach

class Solution:
    def numIslands(self, grid: List[List[str]]) -> int:
        if not grid or not grid[0]:
            return 0
        
        rows = len(grid)
        cols = len(grid[0])
        uf = UnionFind(grid)
        
        for r in range(rows):
            for c in range(cols):
                if grid[r][c] == '1':
                    # Check the neighbor to the right and the neighbor below
                    # As you iterate through the grid, you look at a land cell and its neighbor. If they are both '1', you try to union them.
                    for dr, dc in [(0, 1), (1, 0)]:
                        nr, nc = r + dr, c + dc
                        if 0 <= nr < rows and 0 <= nc < cols and grid[nr][nc] == '1':
                            # "mapping" logic used to flatten a 2D grid into a 1D array.
                            curr_cell_2D_index = r*cols + c
                            neigbor_cell_2D_index = nr*cols + nc
                            uf.union(curr_cell_2D_index, neigbor_cell_2D_index)
        
        return uf.count

class UnionFind:
    def __init__(self, grid):
        row_len, col_len = len(grid), len(grid[0])
        self.parent = [-1] * (row_len * col_len)
        # Rank refers to the depth (height) of the tree.
        self.rank = [0] * (row_len * col_len)
        # represents the number of connected components (disjoint sets / islands) currently in the grid
        # initially we count ever single land cell as it's own tiny isolated island
        # we decrement count by 1 every time we do a union of two land cells as they now form a single bigger island
        self.count = 0
        
        for r in range(row_len):
            for c in range(col_len):
                if grid[r][c] == "1":
                    index = r * col_len + c
                    self.parent[index] = index
                    self.count += 1

    def find(self, i):
        if self.parent[i] != i:
            self.parent[i] = self.find(self.parent[i]) # Path Compression
        return self.parent[i]

    def union(self, i, j):
        root_i = self.find(i)
        root_j = self.find(j)
        if root_i != root_j:
            # Union by Rank
            # Attach the tree with the smaller height to the root of the tree with the larger height, aka make the root of the larger height tree the root of the smaller height tree
            # with this the overall height of the combined tree doesn't change because it will remain the value of the larger height tree
            if self.rank[root_i] > self.rank[root_j]:
                self.parent[root_j] = root_i
            elif self.rank[root_i] < self.rank[root_j]:
                self.parent[root_i] = root_j
            else:
                # does not matter which root becomes the parent as they have the same rank, 
                # the only thing you must ensure is that you increment the rank of the new parent
                # in this case that is root_i
                self.parent[root_j] = root_i
                # Only when merging two trees of equal rank.
                # The height only increases (by 1) if you merge two trees of the same rank.
                self.rank[root_i] += 1
            self.count -= 1

In [ ]:
# Union by Size approach
class UnionFind:
    def __init__(self, grid):
        m, n = len(grid), len(grid[0])
        self.parent = [-1] * (m * n)
        self.size = [0] * (m * n) # Tracks number of nodes in the set
        self.count = 0
        
        for r in range(m):
            for c in range(n):
                if grid[r][c] == "1":
                    index = r * n + c
                    self.parent[index] = index
                    self.size[index] = 1 # Initial size is 1 for each land cell
                    self.count += 1

    def find(self, i):
        if self.parent[i] != i:
            # Path compression: flattens the structure
            self.parent[i] = self.find(self.parent[i])
        return self.parent[i]

    def union(self, i, j):
        root_i = self.find(i)
        root_j = self.find(j)
        
        if root_i != root_j:
            # Union by Size logic
            if self.size[root_i] < self.size[root_j]:
                # Attach smaller tree i under larger tree j
                self.parent[root_i] = root_j
                self.size[root_j] += self.size[root_i]
            else:
                # Attach smaller/equal tree j under larger tree i
                self.parent[root_j] = root_i
                self.size[root_i] += self.size[root_j]
            
            self.count -= 1

class Solution:
    def numIslands(self, grid: List[List[str]]) -> int:
        if not grid or not grid[0]:
            return 0
        
        rows, cols = len(grid), len(grid[0])
        uf = UnionFind(grid)
        
        for r in range(rows):
            for c in range(cols):
                if grid[r][c] == "1":
                    # Check Right and Down neighbors
                    for dr, dc in [(0, 1), (1, 0)]:
                        nr, nc = r + dr, c + dc
                        if 0 <= nr < rows and 0 <= nc < cols and grid[nr][nc] == '1':
                            uf.union(r * cols + c, nr * cols + nc)
        
        return uf.count
    

In [ ]:
# My Union Find Solution
class Solution:
    def numIslands(self, grid: List[List[str]]) -> int:
        if len(grid) == 0:
            return 0

        rowLen = len(grid)
        colLen = len(grid[0])
        self.count = 0

        # initialize a parent array where each element in the 0 indexed matrix
        # is it's own parent
        parent = [i for i in range(rowLen*colLen)]
        rank = [0] * rowLen*colLen

        # initialize minimum island count to the count of all the individual lands in the matrix
        # cause if none of the lands are conncted to each other then we have L islands count total (max count of islands)
        for i in range(rowLen):
            for j in range(colLen):
                if grid[i][j] == '1':
                    self.count += 1

        def find(x):
            if x != parent[x]:
                parent[x] = find(parent[x])

            return parent[x]

        def union(x, y):
            xroot = find(x)
            yroot = find(y)

            if xroot == yroot:
                return
            if rank[xroot] < rank[yroot]:
                xroot, yroot = yroot, xroot

            parent[yroot] = xroot
            rank[xroot] = max(rank[xroot], rank[yroot]+1)

            # If we are joining islands then we are reducing the count of individual islands by 1
            self.count -= 1

        for i in range(rowLen):
            for j in range(colLen):
                if grid[i][j] == '0':
                    continue
                index = i*colLen + j
                if j < colLen-1 and grid[i][j+1] == '1':
                    union(index, index+1)
                if i < rowLen-1 and grid[i+1][j] == '1':
                    union(index, index+colLen)
        return self.count